In [1]:
## DEPENDENCIES
import numpy as np
import sympy as sp
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import fsolve,curve_fit

# 1. Data Loading

In [2]:
df = pd.read_csv('result/only9.5_to10.5.csv')
x_data = df['uM']
y_data = df['Fit']


# df = pd.read_csv('data_for_test/lepidium_metaremoved.csv')
# x_data = df['conc']
# y_data = df['weight']


# df = pd.read_csv('data_for_test/ryegrass_metaremoved.csv')
# x_data = df['conc']
# y_data = df['rootl']

# 2. Model Construction

In [3]:
# Four PL equation function to be used by scipy optimize function
def four_pl(x, A, B, C, D):
    return A + (B - A) / (1 + (x / C) ** D)

A_guess = np.percentile(y_data, 5)  # More robust estimate for lower asymptote then using y_min, not prone to outlier/extreme values
B_guess = np.percentile(y_data, 95)  # More robust estimate for upper asymptote then using y_max, not prone to outlier/extreme values
C_guess = np.median(x_data)  # Still using the median
D_guess = 1  # Hill slope


initial_guess = [A_guess, B_guess, C_guess, D_guess]

# Define bounds for parameters
bounds = ([np.min(y_data) - 1, np.min(y_data), 0, 0], [np.max(y_data) + 1, np.max(y_data), np.max(x_data), 100])
#bounds = ([0, 0, 0, 0], [np.inf, np.inf, np.inf, 10])
# Fit the model to the data
try:
    popt, pcov = curve_fit(four_pl, x_data, y_data, p0=initial_guess, bounds=bounds, maxfev=2000)
    A_fit, B_fit, C_fit, D_fit = popt
    print(f'Fitted Parameters: A={A_fit}, B={B_fit}, C={C_fit}, D={D_fit}')
except Exception as e:
    print(f"An error occurred: {e}")

Fitted Parameters: A=0.09100687634632024, B=1.0100454133183805, C=6.618816246415256, D=1.7705486755903141


# 3. Dose response and confidence interval

In [4]:
def dose_response(A_fit, B_fit, C_fit, D_fit, desired_response, confidence_interval):
    # Calculate desired fraction
    desired_fraction = 1 - desired_response / 100
    
    # Define symbols
    C_fit_sym, A_fit_sym, B_fit_sym, D_fit_sym, desired_fraction_sym = sp.symbols('C_fit A_fit B_fit D_fit desired_fraction')

    # Define the function x
    x_equation = C_fit_sym * (((B_fit_sym - A_fit_sym - desired_fraction_sym * (B_fit_sym - A_fit_sym)) / (desired_fraction_sym * (B_fit_sym - A_fit_sym))) ** (1 / D_fit_sym))

    # calculate the values of x and y
    x = C_fit * (((B_fit - A_fit - desired_fraction * (B_fit - A_fit)) / (desired_fraction * (B_fit - A_fit))) ** (1 / D_fit))
    y = A_fit + (B_fit - A_fit) / (1 + (x / C_fit) ** D_fit) 
    
    # Calculate partial derivatives
    partial_A = sp.diff(x_equation, A_fit_sym)
    partial_B = sp.diff(x_equation, B_fit_sym)
    partial_C = sp.diff(x_equation, C_fit_sym)
    partial_D = sp.diff(x_equation, D_fit_sym)

    # Substitute values into partial derivatives
    eq_to_val = {
        A_fit_sym: A_fit,
        B_fit_sym: B_fit,
        C_fit_sym: C_fit,
        D_fit_sym: D_fit,
        desired_fraction_sym: desired_fraction
    }

    partial_A_value = partial_A.subs(eq_to_val)
    partial_B_value = partial_B.subs(eq_to_val)
    partial_C_value = partial_C.subs(eq_to_val)
    partial_D_value = partial_D.subs(eq_to_val)

    # Compute partial derivatives array
    partial_derivatives = np.array([
        float(partial_A_value),
        float(partial_B_value),
        float(partial_C_value),
        float(partial_D_value)
    ]).reshape(-1, 1)
    
    # calculate the standard deviation
    intermediate_result = pcov @ partial_derivatives
    variance_x = (partial_derivatives.T @ intermediate_result).item()
    std_dev_x = np.sqrt(variance_x)
    
    
    # Compute confidence intervals
    confidence_level = confidence_interval / 100.0
    z_score = norm.ppf((1 + confidence_level) / 2)
    lower_x = x - z_score * std_dev_x
    upper_x = x + z_score * std_dev_x

    return x,y, lower_x, upper_x

response = 50
interval = 95
x, y, lower_x, upper_x = dose_response(A_fit, B_fit, C_fit, D_fit,response, interval)


# Print the results in an intuitive format
print(f"For a {response}% dose response (y = {y:.5f}), the corresponding X value is {x:.5f}.")
print(f"The {interval}% confidence interval for X is [{lower_x:.5f}, {upper_x:.5f}].")

For a 50% dose response (y = 0.55053), the corresponding X value is 6.61882.
The 95% confidence interval for X is [4.35232, 8.88532].
